In [5]:
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [2]:
def gerar_url_voo_azul(mes, dia):
  """Gera a URL de busca para uma data específica."""
  url_modelo = f"https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds=CPV&c[0].std={mes}/{dia}/2025&c[0].as=BSB&p[0].t=ADT&p[0].c=2&p[0].cp=false&f.dl=3&f.dr=3&cc=PTS"
  return url_modelo

In [6]:
def extrair_dados_voo(url):
  """
  Acessa a URL com Selenium e extrai informações dos voos.
  """
  # PARTE 1: Configuração do Selenium
  # Certifique-se de que o caminho para o chromedriver está correto
  service = Service()
  options = webdriver.ChromeOptions()
  # options.add_argument('--headless') # Descomente para rodar sem abrir uma janela do navegador
  driver = webdriver.Chrome(service=service, options=options)

  print(f"Acessando a página: {url}")
  driver.get(url)

  # PARTE 2: Esperar a página carregar
  # Sites de voos carregam preços dinamicamente. Precisamos esperar.
  # Este tempo de espera (15s) é um chute, pode precisar de mais ou menos.
  print("Aguardando o carregamento dos preços...")
  time.sleep(15)

  # PARTE 3: Extração dos Dados
  # Esta é a parte que você precisará adaptar.
  # Você deve usar o "Inspecionar Elemento" do seu navegador para descobrir
  # as classes, IDs ou tags dos elementos que contêm o preço e o horário.

  # Exemplo hipotético de como seria:
  try:
    # Pega todo o HTML da página depois que o JavaScript carregou tudo
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')

    # Encontra todos os 'cards' de voos. A classe '.flight-card' é um EXEMPLO.
    voos = soup.find_all('div', class_=re.compile('flight-card')) # <-- MUDE ESSA CLASSE

    if not voos:
      print("Nenhum voo encontrado. Verifique a classe do container dos voos.")
      print("Dica: A página pode ter um popup ou aviso que impede o carregamento.")

    print(f"Encontrados {len(voos)} voos.\n")

    for voo in voos:
      # Exemplo para extrair horário e preço. As classes são HIPOTÉTICAS.
      # Você PRECISA inspecionar o site para encontrar as classes corretas.
      horario_partida = voo.find('span', class_='departure-time') # <-- MUDE ESSA CLASSE
      preco_pontos = voo.find('span', class_='price-points')     # <-- MUDE ESSA CLASSE

      if horario_partida and preco_pontos:
        print(f"Horário: {horario_partida.text.strip()} | Preço: {preco_pontos.text.strip()} pontos")

  except Exception as e:
    print(f"Ocorreu um erro ao extrair os dados: {e}")

  finally:
    # Fecha o navegador
    driver.quit()

In [9]:
mes_desejado = "10"
dia_desejado = "25"
url_alvo = gerar_url_voo_azul(mes_desejado, dia_desejado)

extrair_dados_voo(url_alvo)

Acessando a página: https://www.voeazul.com.br/br/pt/home/selecao-voo?c[0].ds=CPV&c[0].std=10/25/2025&c[0].as=BSB&p[0].t=ADT&p[0].c=2&p[0].cp=false&f.dl=3&f.dr=3&cc=PTS
Aguardando o carregamento dos preços...
Nenhum voo encontrado. Verifique a classe do container dos voos.
Dica: A página pode ter um popup ou aviso que impede o carregamento.
Encontrados 0 voos.

